# 01 - Data Preprocessing

## A - Libraries

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from metaboDGD.util import data

## B - Retrieving Separate Datasets

In [15]:
# Combine the CAMP cohort datasets to one dataframe
df, cohorts = data.combine_cohort_datasets()

In [16]:
# Convert the df to a numpy array of dim (# samples, # metabolites)
np_df = df.T.to_numpy()[:, :-1].astype(np.float64)

# Get cells that have a 0.0
np_df_zm = (np_df == 0)

# Exponentiate the df by 2
np_exp = np.exp2(np_df)

# Retain the 0.0 values
np_exp[np_df_zm] = 0.0

# Recreate the exponent version of the dataframe
df_rows = list(df.T.index)
df_cols = list(df.T.columns)
df_exp = pd.DataFrame(np_exp, index=df_rows, columns=df_cols[:-1])

In [17]:
dir = 'outputs/'
if not os.path.exists(dir):
    os.makedirs(dir)

df_fname = 'CombinedDataset_CAMP.csv'
df.to_csv(dir + df_fname)

df_exp_fname = 'Exponent_CombinedDataset_CAMP.csv'
df_exp.to_csv(dir + df_exp_fname)

ch_fname = 'cohorts.pkl'
f = open(dir + ch_fname, 'wb')
pickle.dump(cohorts, f)
f.close()

## C - Exploratory Data Analysis

In [ ]:
from metaboDGD.src.prior import SoftballPrior
from torch.distributions.uniform import Uniform

radius=2
softball_prior = SoftballPrior(latent_dim=1, radius=radius, sharpness=1)
sb_samples = softball_prior.sample(n_sample=10000).squeeze(-1).numpy()

uniform_dist = Uniform(low=-radius, high=radius)
un_samples = uniform_dist.sample(sample_shape=(10000,)).numpy()

plt.hist(sb_samples, bins=30, density=True, color='skyblue', edgecolor='black', alpha=0.5)
plt.hist(un_samples, bins=30, density=True, color='pink', edgecolor='black'   , alpha=0.5)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(train_rep.z.detach().numpy())
print(pca.explained_variance_ratio_)
plt.scatter(pca_fit[0:37,0], pca_fit[0:37,1],       label='BRCA1',c='red')
plt.scatter(pca_fit[37:74,0], pca_fit[37:74,1],     label='COAD',c='orange')
plt.scatter(pca_fit[74:93,0], pca_fit[74:93,1],     label='ccRCC3',c='yellow')
plt.scatter(pca_fit[93:124,0], pca_fit[93:124,1],   label='ccRCC4',c='green')
plt.scatter(pca_fit[124:128,0], pca_fit[124:128,1], label='GBM',c='blue')
plt.scatter(pca_fit[128:130,0], pca_fit[128:130,1], label='HurthleCC',c='purple')
plt.scatter(pca_fit[130:139,0], pca_fit[130:139,1], label='PDAC',c='brown')
plt.scatter(pca_fit[139:175,0], pca_fit[139:175,1], label='PRAD',c='black')
plt.xlabel(f'PCA Variance: {(pca.explained_variance_ratio_[0] * 100):.2f}%')
plt.ylabel(f'PCA Variance: {(pca.explained_variance_ratio_[1] * 100):.2f}%')
# plt.legend()
plt.show()

In [ ]:
import scipy.stats as stats

ch = "PRAD"
met_list = list(df.index)
# 9 or -9
IDX = 9
exp_df = df_exp[met_list[IDX]].to_numpy()
log_df = df.T[met_list[IDX]].to_numpy()
dataset_nz = exp_df[exp_df != 0.0]
dataset_nz2 = log_df[log_df != 0.0]

# log_normal_dist = stats.lognorm
# y = log_normal_dist.rvs(std, loc=mean, scale=mean, size=1000)
# x = np.linspace(0, y.max(), 1000)
# param = log_normal_dist.fit(y, floc=0)
# pdf_fitted = log_normal_dist.pdf(x, *param)
#fit
x = np.linspace(-3, np.max(dataset_nz), 10000)
y = np.linspace(-3, np.max(dataset_nz2), 10000)

shape, loc, scale = stats.lognorm.fit(dataset_nz)
pdf               = stats.lognorm.pdf(x, shape, loc=loc, scale=scale)

loc2, scale2 = stats.norm.fit(dataset_nz2)
pdf2         = stats.norm.pdf(y, loc=loc2, scale=scale2)

# plt.plot(x, pdf, c='darkgreen')
# plt.hist(exp_df, bins=30, density=True, color='chartreuse', edgecolor='black', alpha=0.5)
# plt.axvline(loc, c='green', alpha=0.3)

plt.plot(y, pdf2, c='navy')
plt.hist(log_df, bins=30, density=True, color='skyblue', edgecolor='black', alpha=0.5)
# plt.axvline(loc2, c='blue', alpha=0.3)

plt.xlim(left=-3)

plt.xlabel("Log Metabolite Abundance")
plt.ylabel("Density")
# plt.title("LogNormal Distribution Fit - Exp Space")
# plt.xscale('log')
plt.show()

In [ ]:
for i in range(0, df_exp.shape[0]):
    sample_0 = df_exp.to_numpy()[i]

    # sample_0 = cohorts['BRCA1']['matrix'].T.to_numpy()[0]
    sample_0_rev = np.sort(sample_0)[::-1]
    idx = [x for x in range(0, len(sample_0_rev))]

    plt.scatter(idx, sample_0_rev, c='dodgerblue', alpha=0.01)


plt.xscale('log', base=2)
plt.yscale('log', base=2)
# plt.ylim(top=2**11)
plt.title("Metabolite Abundance - Log-Log Plot (All 224 Samples)")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data (could be empirical or synthetic)
x = np.logspace(0.1, 2, 100)
y = x ** -2.5  # example power law: y ∝ x^(-2.5)

# Plot data
plt.figure(figsize=(6, 5))
plt.loglog(x, y, label='Power-law data', marker='o')

# Plot reference line (same slope, arbitrary position)
slope = -2.5
k = 1e2  # constant to adjust vertical placement
x_ref = np.logspace(0.1, 2, 100)
y_ref = k * x_ref ** slope
plt.loglog(x_ref, y_ref, 'k--', label=f'Reference line: slope={slope}')

# Labels and legend
plt.xlabel("x (log scale)")
plt.ylabel("y (log scale)")
plt.title("Log-Log Plot with Power-Law Line")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

# cosine_similarity([np_df[1], dgd_model.dec(train_rep.z)[1].detach().numpy()], dense_output=False)
cosine(np_df[1], dgd_model.dec(train_rep.z)[1].detach().numpy())


In [ ]:
sample = np_df[0]
zero_mask = (sample == 0)
nonzero_mask = ~(zero_mask)

In [ ]:
# test = torch.rand((10,))
# # test[5] = 0.0
# # test[~(test == 0)] = 999.99
# # test
# torch.zeros_like(test)
# torch.exp(gamma_dist.log_prob(torch.full(fill_value=1e-5, size=(1,10))))

gamma_dist = D.Gamma(torch.full(fill_value=10.0, size=(1,)),
                     torch.full(fill_value=0.25, size=(1,)))
plt.hist(gamma_dist.sample(sample_shape=(50000,)).squeeze().numpy(), bins=300)
plt.xlim(left=0, right=100)
plt.show()

In [ ]:
torch.split(torch.Tensor(np_df), 32)

In [ ]:
np.count(np_exp.var(axis=0) > np_exp.mean(axis=0))

In [ ]:
## DO NOT REMOVE
zero_counts = np.count_nonzero(np_exp == 0, axis=0)
plt.scatter(x=np_exp.mean(axis=0),
            y=np_exp.var(axis=0),
            c=zero_counts,
            cmap='viridis',
            marker='o',
            alpha=0.5)
plt.xlabel("Mean Metabolite Abundance (Log 2 Scale)")
plt.ylabel("Variance Metabolite Abundance (Log 2 Scale)")
plt.xscale('log', base=2)
plt.yscale('log', base=2)
plt.colorbar(label='Number of Zeroes')
plt.title("Mean-Variance Plot")

In [ ]:
type(np_df == 0)

In [ ]:
## DO NOT REMOVE
plt.hist(
    np_exp[:,100],
)
plt.title("Histogram of Processed Metabolite Abundances")
plt.show()

In [ ]:
# BRCA1 ccRCC3  ccRCC4  COAD    GBM HurthleCC   PDAC    PRAD
target_cohort = "BRCA1"
START_IDX = len(cohorts["ccRCC3"]['sample_list']) * 0
END_IDX = len(cohorts[target_cohort]['sample_list'])
cohort_df = np_df[START_IDX:START_IDX+END_IDX]
zero_counts = np.count_nonzero(cohort_df == 0, axis=0)

plt.scatter(x=cohort_df.mean(axis=0),
            y=cohort_df.var(axis=0),
            c=zero_counts,
            cmap='viridis',
            marker='o',
            alpha=0.3)
plt.xlabel("Mean Metabolite Abundance (Log 2)")
plt.ylabel("Variance Metabolite Abundance (Log 2)")
plt.title(target_cohort)
plt.colorbar(label='Number of Zeroes')

In [ ]:
# Gets the indices of the 26 metabolites present across all samples (26)
# set([x for x in range(0,1915)]) - set(np.unique(np.where(np_df == 0)[1]))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
np_df = df.T.to_numpy()
pca_model = PCA(n_components=2)
results = pca_model.fit_transform(np_df)
print(pca_model.explained_variance_ratio_)
plt.scatter(results[0:47,0], results[0:47, 1], c='red'          ,label="BRCA1")
plt.scatter(results[47:86,0], results[47:86, 1], c='orange'     ,label="COAD")
plt.scatter(results[86:133,0], results[86:133, 1], c='yellow'   ,label="ccRCC3")
plt.scatter(results[133:157,0], results[133:157, 1], c='green'  ,label="ccRCC4")
plt.scatter(results[157:163,0], results[157:163, 1], c='blue'   ,label="GBM")
plt.scatter(results[163:166,0], results[163:166, 1], c='purple' ,label="HurthleCC")
plt.scatter(results[166:178,0], results[166:178, 1], c='pink'   ,label="PDAC")
plt.scatter(results[178:224,0], results[178:224, 1], c='black'  ,label="PRAD")
plt.legend()
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=8)
gmm = gmm.fit(np_df)

In [ ]:
results = gmm.sample(n_samples=1000)

In [ ]:
import pandas as pd
pd.Series(results[1]).value_counts()

In [ ]:
pca_new = PCA(n_components=2)
results_new = pca_model.transform(results[0])
# results_new = pca_model.transform(results[0])
# plt.scatter(results_new[0:224,0]   , results_new[0:224, 1], c='red'          ,label="BRCA1")
# plt.scatter(results_new[224:426,0]  , results_new[224:426, 1], c='orange'     ,label="COAD")
# plt.scatter(results_new[426:609,0] , results_new[426:609, 1], c='yellow'   ,label="ccRCC3")
# plt.scatter(results_new[609:785,0], results_new[609:785, 1], c='green'  ,label="ccRCC4")
# plt.scatter(results_new[785:903,0], results_new[785:903, 1], c='blue'   ,label="GBM")
# plt.scatter(results_new[903:956,0], results_new[903:956, 1], c='purple' ,label="HurthleCC")
# plt.scatter(results_new[956:983,0], results_new[956:983, 1], c='pink'   ,label="PDAC")
plt.scatter(results_new[983:1000,0], results_new[983:1000, 1], c='black'  ,label="PRAD")
plt.legend()
plt.show()

In [ ]:
# pca_new = PCA(n_components=2)
results_new = pca_model.transform(results[0])
# plt.scatter(results_new[0:47,0]   , results_new[0:47, 1], c='red'          ,label="BRCA1")
# plt.scatter(results_new[47:86,0]  , results_new[47:86, 1], c='orange'     ,label="COAD")
# plt.scatter(results_new[86:133,0] , results_new[86:133, 1], c='yellow'   ,label="ccRCC3")
# plt.scatter(results_new[133:157,0], results_new[133:157, 1], c='green'  ,label="ccRCC4")
# plt.scatter(results_new[157:163,0], results_new[157:163, 1], c='blue'   ,label="GBM")
# plt.scatter(results_new[163:166,0], results_new[163:166, 1], c='purple' ,label="HurthleCC")
# plt.scatter(results_new[166:178,0], results_new[166:178, 1], c='pink'   ,label="PDAC")
# plt.scatter(results_new[178:224,0], results_new[178:224, 1], c='black'  ,label="PRAD")
plt.legend()
plt.show()

In [ ]:
# Get the xls file
cohort_name = "PRAD"
xls = pd.ExcelFile(f'data/PreprocessedData_{cohort_name}.xlsx')

# Get the dataframes for the preprocessed metabolomics data
# t = xls.parse("metabo_imputed_filtered_Tumor")
n = xls.parse("metabo_imputed_filtered_Normal")

# Replace IDs of the dataframes
# t.rename({"Unnamed: 0": "t_met"}, inplace=True, axis=1)
n.rename({"Unnamed: 0": "n_met"}, inplace=True, axis=1)

# Get list of metabolites
# t_list = t["t_met"].to_list()
n_list = n["n_met"].to_list()
# met_list = list(set(t_list) | set(n_list))

# Create a dictionary of the metabolite names and HMDB IDs
###########
# 05/25/25 - Eliminated HMDB IDs use because only 72% of the features have HMDB IDs across all cohorts
# For Reference:
#           TOTAL_FEATURES  TOTAL_FEATURES_WITH_HMDB_IDS
# BRCA1     324             215
# COAD      160	            141
# ccRCC3    727	            551
# ccRCC4    951	            701
# GBM       704	            357
# HurthleCC 668	            523
# PDAC      325	            279
# PRAD      382	            320
###########
# metanno = xls.parse("metanno")
# metanno_dict = metanno.set_index("H_name")["H_HMDB"].to_dict()
# hmdb_list = set(metanno.loc[metanno["H_name"].isin(n_list)]["H_HMDB"].tolist()


# Get dataframe with only sorted shared metabolites
# t_shared = t[t["t_met"].isin(met_list)]
# t_shared.sort_values("t_met", ignore_index=True, inplace=True)
# n_shared = n[n["n_met"].isin(met_list)]
# n_shared.sort_values("n_met", ignore_index=True, inplace=True)
# merged = pd.concat([t_shared, n_shared], axis=1)

# n = n.set_index(n["n_met"])
n_no_labels = n.drop(labels=['n_met'], axis=1)
print(n.set_index("n_met"))